# 生成上下文

pipeline

主要思路：
1. 输出的上下文格式为：{user id}会在{time/time period}去{grid}。也就是用户1将会在（一个精确时间）或者（一个模糊的时间段）访问（grid）。
   1. User {userID} will go to the area with grid number {grid number} at {time/time period}.
2. {time/time period}是随机对未来某个stay的进行说明。
3. 已经给出了上下文信息的停留点不会重复给出上下文。（没有处理改变意图的情况）
4. 对于出现频率最高的3个停留点不会给出上下文。这对应的解释是长期有规律的活动不需要上下文。

---

代码流程：
1. 获取整个数据的行号列表，记为stay_list。
2. 获取出现频率最高的N个停留点，作为周期停留点。将这些停留点对应的行号从行号列表中删除，得到**非周期且未生成上下文**停留点对应的行号列表，记为 aperiodic_stay_list 。
3. 按**随机**遍历 aperiodic_stay_list 列表。（因为有很多周期性的停留点，也就是理论上周期性的停留点的数量是大于非周期的停留点的。如果按顺序遍历的话，有可能导致还有上下文的停留都集中在数据的前面了）
4. 选择一个停留点的行号，记为 current_stay 。
5. 从 current_stay 之后的行号中选一个准备生成该行号的对应停留点的上下文，被选中的停留点记为 generate_context_stay 。
6. 从 aperiodic_stay_list 中随机选一个停留点。计算 generate_context_stay 与 current_stay 的时间差。
7. 如果时间差大于一个月。生成一个[0,1]的随机数，记为 year_random 。
   1. 如果 year_random 大于或者等于0.1，则生成一个包含按月模糊时间的上下文。
   2. 如果 year_random 小于0.1，则生成一个包含准确时间的上下文。
   3. 将生成的上下文内容填入 current_stay 的 context 字段中。
8. 如果时间差小于一个月，且大于一个星期，生成一个[0,1]的随机数，记为 month_random 。
   1. 如果 month_random 大于或者等于0.3，则生成一个包含按周模糊的上下文。
   2. 如果 month_random 小于0.3，则生成一个包含准确时间的上下文。
   3. 将生成的上下文内容填入 current_stay 的 context 字段中。
9. 如果时间差小于一个星期，且大于一天，生成一个[0,1]的随机数，记为 week_random 。
   1. 如果 week_random 大于或者等于0.5，则生成一个包含按天模糊的上下文。
   2. 如果 week_random 小于0.5，则生成一个包含准确时间的上下文。
   3. 将生成的上下文内容填入 current_stay 的 context 字段中。
10. 如果时间差小于一天，生成一个[0,1]的随机数，记为 day_random 。
   1. 如果 day_random 大于或者等于0.7，则生成一个包含按小时模糊的上下文。
   2. 如果 day_random 小于0.7，则生成一个包含准确时间的上下文。
   3. 将生成的上下文内容填入 current_stay 的 context 字段中。
11. 将 generate_context_stay 从 aperiodic_stay_list 中删除。
12. 直到数据的最后一个行或者 aperiodic_stay_list 中未空。


## 

In [13]:
import pandas as pd
from io import StringIO
import random
from datetime import timedelta
import numpy as np

# --- 常量定义 ---
TIME_LIMITS = {
    'month': timedelta(days=30),
    'week': timedelta(days=7),
    'day': timedelta(days=1)
}
N_TOP_FREQUENT = 3 # 排除的周期停留点数量


# # --- 模拟您的输入数据 ---
# data = """,userID,stime,etime,lon,lat,duration,grid
# 0,173,2007-11-30 14:36:34,2007-11-30 18:18:47,116.3300116,39.975275,13333.0,14994
# 47,173,2007-11-30 18:40:25,2007-12-01 10:04:59,116.3132266,39.9703666,55474.0,14616
# 206,173,2007-12-01 10:23:33,2007-12-01 11:02:19,116.410585,39.96704,2326.0,16127
# 213,173,2007-12-01 11:02:19,2007-12-02 13:36:08,116.41666,39.8555383,95629.0,16304
# 617,173,2007-12-02 14:18:55,2007-12-02 18:39:35,116.313185,39.9660799,15640.0,14615
# 650,173,2007-12-02 18:51:50,2007-12-02 19:46:12,116.3198316,39.9465316,3262.0,14802
# 684,173,2007-12-02 20:00:44,2007-12-04 16:43:35,116.31226,39.9692849,160971.0,14616
# 811,173,2007-12-15 14:09:32,2007-12-16 20:01:27,116.3131133,39.9684866,107515.0,14616
# 859,173,2007-12-16 20:01:27,2007-12-16 21:49:33,116.3117599,39.97306,6486.0,14616
# """

# # 1. 数据加载与预处理
# df = pd.read_csv(StringIO(data), index_col=0)

userid = '002'
df = pd.read_csv('./Data/Output/Stays/{}.csv'.format(userid), index_col=0)

df['stime'] = pd.to_datetime(df['stime'])
df['etime'] = pd.to_datetime(df['etime'])
df['context'] = None # 初始化 context 列

# 2. 识别周期停留点并构建 aperiodic_stay_list
stay_list = list(df.index)

# 获取频率最高的 N 个 grid
top_n_grids = df['grid'].value_counts().nlargest(N_TOP_FREQUENT).index.tolist()

# 获取所有非周期停留点的索引列表 (这些行可以作为目标 generate_context_stay)
aperiodic_stay_list = df[~df['grid'].isin(top_n_grids)].index.tolist()

# 过滤掉最后一个停留点，它不能作为起点 current_stay
# 确保所有索引都用于后续操作
all_indices = df.index.tolist()
print(f"原始数据行号 (Index): {all_indices}")
print(f"非周期目的地行号 (Aperiodic): {aperiodic_stay_list}")
print(f"Top {N_TOP_FREQUENT} 周期 Grid: {top_n_grids}\n")

# --- 辅助函数：模糊时间生成 ---
def get_fuzzy_time_expression(time_delta, target_time):
    """根据时间差和目标时间生成模糊时间表达式。"""
    
    # 按月模糊
    if time_delta > TIME_LIMITS['month']:
        months = round(time_delta.days / 30)
        return f"in about {months} months" if months > 1 else "in the next month"
    
    # 按周模糊
    elif time_delta > TIME_LIMITS['week']:
        weeks = round(time_delta.days / 7)
        weekday = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"][target_time.weekday()]
        return f"in about {weeks} weeks, specifically next {weekday}"
    
    # 按天模糊
    elif time_delta > TIME_LIMITS['day']:
        days = time_delta.days
        return f"in about {days} days, on {target_time.date().strftime('%Y-%m-%d')}"
    
    # 按小时模糊
    else: # time_delta <= 1 day
        hour = target_time.hour
        if 5 <= hour < 12:
            period = "in the morning"
        elif 12 <= hour < 18:
            period = "in the afternoon"
        elif 18 <= hour < 22:
            period = "in the evening"
        else:
            period = "late at night"
        return f"later today, {period}"

# --- 主要循环逻辑 (步骤 3 - 12) ---
# 3. 按随机遍历 aperiodic_stay_list 列表（作为未来的目标点）
random.shuffle(aperiodic_stay_list)

context_generated_count = 0
current_stay_candidates = df.index[:-1].tolist() # 所有非最后一个行号都可以作为起点

while aperiodic_stay_list and current_stay_candidates:
    # 6. 从 current_stay_candidates 中随机选一个起点 current_stay
    #    (注: 原始逻辑说从 aperiodic_stay_list 随机选，但 aperiodic_stay_list 是目标点，
    #     起点应从所有非终点行号中选，这里做合理修正)
    current_stay_idx = random.choice(current_stay_candidates)
    
    # 5. 从 current_stay 之后的行号中选一个目标点 generate_context_stay
    #    要求目标点必须在 aperiodic_stay_list 中（未被预测过，且非周期）
    
    # 筛选出满足条件的未来目标点
    future_aperiodic_stays = [
        target_idx for target_idx in aperiodic_stay_list 
        if target_idx > current_stay_idx
    ]
    
    if not future_aperiodic_stays:
        # 如果找不到合适的未来目标，将当前起点从候选列表中删除，避免死循环
        current_stay_candidates.remove(current_stay_idx)
        continue

    # 随机选择一个目标点
    generate_context_stay_idx = random.choice(future_aperiodic_stays)
    
    # 获取停留点数据
    current_stay = df.loc[current_stay_idx]
    generate_context_stay = df.loc[generate_context_stay_idx]
    
    start_grid = current_stay['grid']
    end_grid = generate_context_stay['grid']
    user_id = current_stay['userID']
    
    # 计算时间差 (从当前停留点的结束时间到目标停留点的开始时间)
    time_delta = generate_context_stay['stime'] - current_stay['etime']
    
    # 初始化时间表达式为 None
    time_expression = None
    is_accurate_time = False

    # --- 7. 时间差大于一个月 (year_random) ---
    if time_delta > TIME_LIMITS['month']:
        year_random = random.random()
        if year_random >= 0.1:
            time_expression = get_fuzzy_time_expression(time_delta, generate_context_stay['stime']) # 7.1 按月模糊
        else:
            is_accurate_time = True # 7.2 准确时间

    # --- 8. 时间差小于一个月，大于一个星期 (month_random) ---
    elif time_delta > TIME_LIMITS['week']:
        month_random = random.random()
        if month_random >= 0.3:
            time_expression = get_fuzzy_time_expression(time_delta, generate_context_stay['stime']) # 8.1 按周模糊
        else:
            is_accurate_time = True # 8.2 准确时间

    # --- 9. 时间差小于一个星期，大于一天 (week_random) ---
    elif time_delta > TIME_LIMITS['day']:
        week_random = random.random()
        if week_random >= 0.5:
            time_expression = get_fuzzy_time_expression(time_delta, generate_context_stay['stime']) # 9.1 按天模糊
        else:
            is_accurate_time = True # 9.2 准确时间

    # --- 10. 时间差小于一天 (day_random) ---
    else:
        day_random = random.random()
        if day_random >= 0.7:
            time_expression = get_fuzzy_time_expression(time_delta, generate_context_stay['stime']) # 10.1 按小时模糊
        else:
            is_accurate_time = True # 10.2 准确时间

    # 构造最终时间表达式
    if is_accurate_time:
        time_expression = generate_context_stay['stime'].strftime('%Y-%m-%d %H:%M:%S')

    # 构造上下文文本
    context_text = (
        f"User {user_id} will move from grid {start_grid} to grid {end_grid}, "
        f"arriving around {time_expression}."
    )

    # 7.3, 8.3, 9.3, 10.3: 将生成的上下文内容填入 current_stay 的 context 字段中
    df.loc[current_stay_idx, 'context'] = context_text
    context_generated_count += 1
    
    # 11. 将 generate_context_stay 从 aperiodic_stay_list 中删除
    aperiodic_stay_list.remove(generate_context_stay_idx)
    
    # 目标行号被预测后，它不能再次作为目标，也不能再作为起点（如果它被预测了，就不需要它再做预测了）
    # 确保 current_stay 只能生成一次上下文 (否则 context 字段会被覆盖)
    current_stay_candidates.remove(current_stay_idx)

# 12. 循环直到数据的最后一个行或者 aperiodic_stay_list 中未空。
#     循环结束条件已通过 while 语句控制。


# --- 结果展示 ---
print("\n--- 基于详细逻辑的上下文生成结果 ---")
print(f"总共生成了 {context_generated_count} 条上下文。")
print("原始数据 (仅显示关键列和生成的 context):")
print(df[['userID', 'stime', 'etime', 'grid', 'context']].head(len(df)))

df.to_csv('./Data/Output/Context/{}.csv'.format(userid))

原始数据行号 (Index): [0, 1034, 1926, 2326, 2796, 4221, 4328, 5640, 6471, 6681, 6708, 6844, 7058, 8716, 8836, 9870, 10289, 10672, 11186, 12921, 15851, 16432, 16658, 18548, 18889, 19207, 19382, 19491, 20612, 20934, 21800, 22027, 22078, 22344, 23491, 23664, 24053, 25126, 25440, 26268, 26587, 26846, 27916, 28837, 29595, 30163, 30866, 32382, 36213, 36814, 37484, 37917, 38259, 38750, 39176, 40121, 41025, 41402, 41554, 42253, 42640, 43312, 43428, 43700, 44512, 45509, 45830, 45978, 46264, 46343, 46695, 47552, 47870, 48276, 48435, 49050, 49091, 49404, 51207, 51913, 53745, 55075, 55420, 55761, 55763, 55786, 56531, 56934, 57485, 57980, 58503, 58600, 59040, 59268, 61946, 62103, 62538, 63172, 63752, 64225, 64534, 64810, 65058, 66139, 67207, 67747, 68019, 69123, 70219, 71356, 72414, 73408, 73990, 74156, 74404, 75172, 75584, 76563, 77095, 77643, 78848, 79004, 79034, 79543, 80135, 81054, 81508, 82170, 82445, 82519, 82911, 83096, 84396, 84599, 85243, 85564, 86101, 86246, 86622, 87169, 87518, 87651, 88437, 8

## 之前试验代码


### （流程有问题）生成的上下文时间是完全准确的

In [ ]:
import pandas as pd
from io import StringIO
import numpy as np

# 模拟您的输入数据
# data = """,userID,stime,etime,lon,lat,duration,grid
# 0,173,2007-11-30 14:36:34,2007-11-30 18:18:47,116.3300116,39.975275,13333.0,14994
# 47,173,2007-11-30 18:40:25,2007-12-01 10:04:59,116.3132266,39.9703666,55474.0,14616
# 206,173,2007-12-01 10:23:33,2007-12-01 11:02:19,116.410585,39.96704,2326.0,16127
# 213,173,2007-12-01 11:02:19,2007-12-02 13:36:08,116.41666,39.8555383,95629.0,16304
# 617,173,2007-12-02 14:18:55,2007-12-02 18:39:35,116.313185,39.9660799,15640.0,14615
# 650,173,2007-12-02 18:51:50,2007-12-02 19:46:12,116.3198316,39.9465316,3262.0,14802
# 684,173,2007-12-02 20:00:44,2007-12-04 16:43:35,116.31226,39.9692849,160971.0,14616
# 811,173,2007-12-15 14:09:32,2007-12-16 20:01:27,116.3131133,39.9684866,107515.0,14616
# 859,173,2007-12-16 20:01:27,2007-12-16 21:49:33,116.3117599,39.97306,6486.0,14616
# """

# 加载数据
# df = pd.read_csv(StringIO(data), index_col=0)
userid = '002'
df = pd.read_csv('./Data/Output/Stays/{}.csv'.format(userid), index_col=0)

# 1. 数据类型转换
df['stime'] = pd.to_datetime(df['stime'])
df['etime'] = pd.to_datetime(df['etime'])

# 2. 识别出现频率最高的 3 个 grid
# nlargest(3) 可能会因为样本量小而包含多个频率相同的值，但这里只取前3个。
top_3_grids = df['grid'].value_counts().nlargest(3).index.tolist()
print(f"出现频率最高的 3 个 Grid: {top_3_grids}\n")

# 3. 准备下一个停留点的信息（时间、网格）
# 使用 shift(-1) 将下一行的数据填充到当前行
df['next_stime'] = df['stime'].shift(-1)
df['next_grid'] = df['grid'].shift(-1)

# 4. 初始化 context 列
df['context'] = None

# 5. 基于条件生成 context
def generate_context(row):
    # 规则 6: 最后一个停留点没有下一个目的地，不生成上下文
    if pd.isna(row['next_stime']):
        return None

    # 规则 7: 如果下一个目的地（next_grid）属于周期性活动（Top 3），则不生成上下文
    # 这里的预测是“用户将前往 {next_grid}”，如果 {next_grid} 是周期性地点，则跳过
    if int(row['next_grid']) in top_3_grids:
        return None

    # 规则 3, 4, 5: 构造文本
    user_id = row['userID']
    # {time/time period} 使用下一个停留点的准确开始时间
    time_str = row['next_stime'].strftime('%Y-%m-%d %H:%M:%S')
    grid_number = int(row['next_grid'])

    # 规则 2: 文本格式
    context_text = f"User {user_id} will go to the area with grid number {grid_number} at {time_str}."
    
    return context_text

# 应用函数生成 context
df['context'] = df.apply(generate_context, axis=1)

# 6. 清理辅助列并显示结果
df = df.drop(columns=['next_stime', 'next_grid'])
print("--- 最终结果 (新增 context 列) ---")
print(df[['userID', 'stime', 'grid', 'context']])

df.to_csv('./Data/Output/Context/{}.csv'.format(userid))


出现频率最高的 3 个 Grid: [14989, 15742, 12907]

--- 最终结果 (新增 context 列) ---
        userID               stime   grid  \
0            2 2008-10-23 12:45:24  14989   
1034         2 2008-10-23 14:53:33  14989   
1926         2 2008-10-23 16:40:41  14989   
2326         2 2008-10-24 00:51:26  15931   
2796         2 2008-10-24 04:17:29  15742   
...        ...                 ...    ...   
243232       2 2009-03-19 14:46:29  14989   
243817       2 2009-03-21 02:33:04  15365   
245255       2 2009-03-21 12:09:51  14989   
245296       2 2009-03-22 04:01:23  12918   
246890       2 2009-03-22 06:32:18  12918   

                                                  context  
0                                                    None  
1034                                                 None  
1926    User 2 will go to the area with grid number 15...  
2326                                                 None  
2796    User 2 will go to the area with grid number 14...  
...                           

In [5]:
print(df[['context']])

                                                  context
0                                                    None
1034                                                 None
1926    User 2 will go to the area with grid number 15...
2326                                                 None
2796    User 2 will go to the area with grid number 14...
...                                                   ...
243232  User 2 will go to the area with grid number 15...
243817                                               None
245255  User 2 will go to the area with grid number 12...
245296  User 2 will go to the area with grid number 12...
246890                                               None

[341 rows x 1 columns]


### （流程有问题）生成的时间是一个模糊的

In [8]:
import pandas as pd
from io import StringIO
import random
from datetime import timedelta

# 模拟您的输入数据
data = """,userID,stime,etime,lon,lat,duration,grid
0,173,2007-11-30 14:36:34,2007-11-30 18:18:47,116.3300116,39.975275,13333.0,14994
47,173,2007-11-30 18:40:25,2007-12-01 10:04:59,116.3132266,39.9703666,55474.0,14616
206,173,2007-12-01 10:23:33,2007-12-01 11:02:19,116.410585,39.96704,2326.0,16127
213,173,2007-12-01 11:02:19,2007-12-02 13:36:08,116.41666,39.8555383,95629.0,16304
617,173,2007-12-02 14:18:55,2007-12-02 18:39:35,116.313185,39.9660799,15640.0,14615
650,173,2007-12-02 18:51:50,2007-12-02 19:46:12,116.3198316,39.9465316,3262.0,14802
684,173,2007-12-02 20:00:44,2007-12-04 16:43:35,116.31226,39.9692849,160971.0,14616
811,173,2007-12-15 14:09:32,2007-12-16 20:01:27,116.3131133,39.9684866,107515.0,14616
859,173,2007-12-16 20:01:27,2007-12-16 21:49:33,116.3117599,39.97306,6486.0,14616
"""

# 1. 数据加载与预处理
df = pd.read_csv(StringIO(data), index_col=0)


# userid = '002'
# df = pd.read_csv('./Data/Output/Stays/{}.csv'.format(userid), index_col=0)

df['stime'] = pd.to_datetime(df['stime'])
df['etime'] = pd.to_datetime(df['etime'])

# 识别出现频率最高的 3 个 grid
top_3_grids = df['grid'].value_counts().nlargest(3).index.tolist()

# 2. 准备下一个停留点的信息
df['next_stime'] = df['stime'].shift(-1)
df['next_grid'] = df['grid'].shift(-1)
df['start_grid'] = df['grid'] # 当前停留点作为起点

# 3. 定义随机时间生成函数
def get_random_time_expression(target_time, reference_time):
    """根据随机策略将精确时间转换为精确或模糊的文本。"""
    
    # 策略 1: 精确时间 (30% 概率)
    if random.random() < 0.3:
        return target_time.strftime('%Y-%m-%d %H:%M:%S')

    # 策略 2: 模糊日期（基于周和天）(40% 概率)
    elif random.random() < 0.7:
        time_delta = target_time - reference_time
        
        if time_delta.total_seconds() < 3600 * 24: # 24小时内
            return "later today" # 今天晚些时候
        elif time_delta.days < 7: # 1周内
            weekday = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"][target_time.weekday()]
            # 随机选择 'this' 或 'next'
            if time_delta.days > 2:
                return f"next {weekday}"
            else:
                return f"this {weekday}"
        elif time_delta.days < 30: # 1个月内
             return f"in about {round(time_delta.days / 7)} weeks"
        else:
             return "in the near future" # 较远的未来
    
    # 策略 3: 模糊时段（基于一天的时间）(30% 概率)
    else:
        hour = target_time.hour

        if 5 <= hour < 12:
            return "in the morning"
        elif 12 <= hour < 14:
            return "in the noon"
        elif 14 <= hour < 18:
            return "in the afternoon"
        elif 18 <= hour < 22:
            return "in the evening"
        else:
            return "late at night"

        # if 5 <= hour < 12:
        #     return "in the morning (上午)"
        # elif 12 <= hour < 14:
        #     return "in the noon (中午)"
        # elif 14 <= hour < 18:
        #     return "in the afternoon (下午)"
        # elif 18 <= hour < 22:
        #     return "in the evening (傍晚)"
        # else:
        #     return "late at night (深夜)"


# 4. 生成 context
def generate_improved_context(row):
    # 检查是否是最后一个停留点
    if pd.isna(row['next_stime']):
        return None

    # 检查目的地是否是周期性活动 (Top 3)
    if int(row['next_grid']) in top_3_grids:
        return None

    # 获取随机时间表达式
    random_time_expression = get_random_time_expression(row['next_stime'], row['etime'])
    
    start_grid = int(row['start_grid'])
    end_grid = int(row['next_grid'])
    user_id = row['userID']
    
    # 构造新的上下文文本
    context_text = (
        f"User {user_id} will move from grid {start_grid} to grid {end_grid}, "
        f"arriving around {random_time_expression}."
    )
    
    return context_text

# 应用函数并清理
df['improved_context'] = df.apply(generate_improved_context, axis=1)
print(df)

# 清理辅助列并显示结果
df = df.drop(columns=['next_stime', 'next_grid', 'start_grid'])

print("--- 随机时间上下文生成结果 ---")
# 为了展示对比，这里只显示起点、终点和新的上下文列
print(df[['userID', 'stime', 'etime', 'grid', 'improved_context']])

# df.to_csv('./Data/Output/ImprovedContext/{}.csv'.format(userid))

     userID               stime               etime         lon        lat  \
0       173 2007-11-30 14:36:34 2007-11-30 18:18:47  116.330012  39.975275   
47      173 2007-11-30 18:40:25 2007-12-01 10:04:59  116.313227  39.970367   
206     173 2007-12-01 10:23:33 2007-12-01 11:02:19  116.410585  39.967040   
213     173 2007-12-01 11:02:19 2007-12-02 13:36:08  116.416660  39.855538   
617     173 2007-12-02 14:18:55 2007-12-02 18:39:35  116.313185  39.966080   
650     173 2007-12-02 18:51:50 2007-12-02 19:46:12  116.319832  39.946532   
684     173 2007-12-02 20:00:44 2007-12-04 16:43:35  116.312260  39.969285   
811     173 2007-12-15 14:09:32 2007-12-16 20:01:27  116.313113  39.968487   
859     173 2007-12-16 20:01:27 2007-12-16 21:49:33  116.311760  39.973060   

     duration   grid          next_stime  next_grid  start_grid  \
0     13333.0  14994 2007-11-30 18:40:25    14616.0       14994   
47    55474.0  14616 2007-12-01 10:23:33    16127.0       14616   
206    2326.0  161

In [4]:
df[['improved_context']]

,improved_context
0,None
47,None
206,User 173 will move from grid 16127 to grid 163...
213,User 173 will move from grid 16304 to grid 146...
617,User 173 will move from grid 14615 to grid 148...
650,None
684,None
811,None
859,None
